# Method 1 - Using iNaturalist-pretrained ResNet-50

In [3]:
import random
random.seed(43)

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [2]:
from DeformableResNet import ResNet_AvgPool_classifier, Bottleneck

In [4]:
concat = lambda x: np.concatenate(x, axis=0)
to_np  = lambda x: x.data.to('cpu').numpy()

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
val_dataset_transform = transforms.Compose(
  [transforms.Resize(256), 
  transforms.CenterCrop(224), 
  transforms.ToTensor(), 
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [7]:
validation_folder = ImageFolder(root='/home/mohammad/datasets/CUB/test0/', transform=val_dataset_transform)
val_loader        = DataLoader(validation_folder, batch_size=512, shuffle=False, num_workers=8, pin_memory=False)

## iNAT ResNet-50 

In [8]:
inat_resnet = ResNet_AvgPool_classifier(Bottleneck, [3, 4, 6, 4])
my_model_state_dict = torch.load('./Forzen_Method1-iNaturalist_avgpool_200way1_85.83_Manuscript.pth')
inat_resnet.load_state_dict(my_model_state_dict, strict=True)

<All keys matched successfully>

In [9]:
# Dimension of classification head
print(list(inat_resnet.parameters())[-2].shape)

torch.Size([200, 2048])


In [10]:
# Freeze backbone (for training only)
for param in list(inat_resnet.parameters())[:-2]:
  param.requires_grad = False
    
# to CUDA
inat_resnet.to(device)

resnet53_features

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(inat_resnet.classifier.parameters())

In [12]:
def test_cub(model):
  model.eval()
  
  running_loss = 0.0
  running_corrects = 0
  
  predictions = []
  confidence = []
  
  with torch.inference_mode():
    for _, (data, target) in enumerate(val_loader):
      data   = data.to(device)
      target = target.to(device)
      outputs = model(data)
      loss = criterion(outputs, target)
      _, preds = torch.max(outputs, 1)
      probs, _ = torch.max(F.softmax(outputs, dim=1), 1)
      running_loss += loss.item() * target.size(0)
      running_corrects += torch.sum(preds == target.data)
      
      predictions.extend(preds.data.cpu().numpy())
      confidence.extend((probs.data.cpu().numpy()*100).astype(np.int32))

  epoch_loss = running_loss / len(validation_folder)
  epoch_acc = running_corrects.double() / len(validation_folder)

  print('-' * 10)
  print('loss: {:.4f}, acc: {:.4f}'.format(epoch_loss, 100*epoch_acc))
  
  return predictions, confidence

In [13]:
cub_test_preds, cub_test_confs = test_cub(inat_resnet)

----------
loss: 0.5511, acc: 85.8302
